In [4]:
from sklearn import datasets
import numpy as np

iris = datasets.load_iris()

X = iris.data
y = iris.target

print(X)
print(y)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [12]:
attack_target = 1
# 攻击的目标类别

mean_features = np.mean(X[y == attack_target], axis = 0)
#求出中毒的样本特征向量的均值

print(mean_features)

np.random.seed(6)
# 为了复现结果， 设置随机种子

print(mean_features + np.random.normal(0, 0.1, size = (5, X.shape[1])))

poisoning_samples = mean_features + np.random.normal(0, 0.1, size = (5, X.shape[1]))
# 这些生成出来的例子是有毒例子，它们的特征会让算法认为是第1种鸢尾花
# 而后续将他们的标签设置为别的种类

poisoning_labels = np.array([0, 2, 0, 2, 0])
# 设置其它的标签

[5.936 2.77  4.26  1.326]
[[5.90482163 2.84290039 4.28178208 1.23609082]
 [5.68732193 2.86132515 4.37270637 1.17459068]
 [6.09992911 2.72701064 4.52312806 1.38618222]
 [5.90241184 2.89377378 4.27111282 1.33891512]
 [5.94361276 2.75448718 4.32342253 1.4070655 ]]


In [14]:
X_poisoned = np.vstack((X, poisoning_samples))
y_poisoned = np.append(y, poisoning_labels)

# 投毒

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_poisoned, y_poisoned, test_size = 0.25, random_state = 0)
# 划分训练集和测试集 比例为 3：1

clf = RandomForestClassifier(random_state = 0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# 使用随机森林分类

print(f"Accuracy after poisoning: {accuracy_score(y_test, y_pred)}")

Accuracy after poisoning: 0.8717948717948718


In [ ]:
# 刚刚训练出来的是含有有毒数据的模型
# 再重新训练一个没毒的模型
# 再把二者比较一下

In [17]:
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X, y, test_size = 0.25, random_state = 0)

clf_o = RandomForestClassifier(random_state = 0)
clf_o.fit(X_train_o, y_train_o)

y_pred_o = clf_o.predict(X_test_o)

print(f"Accuracy before poisoning: {accuracy_score(y_test_o, y_pred_o)}")

Accuracy before poisoning: 0.9736842105263158
